In [3]:
# Charger les librairie nécessaire .
from bs4 import BeautifulSoup
import lxml
import requests
import pandas as pd
import numpy as np
import nltk
import string
import fasttext
import contractions
from matplotlib import pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
#plt.xticks(rotation=70)
#pd.options.mode.chained_assignment = None
#pd.set_option('display.max_colwidth', 100)
#%matplotlib inline
#pip install fasttext_win
#pip install contractions
#nltk.download('averaged_perceptron_tagger')


In [ ]:
def ratin(i):
        switcher={
                "['result-rating', 'one']":'1',
                "['result-rating', 'one', 'half']":'1.5',
                "['result-rating', 'two']":'2',
                "['result-rating', 'two', 'half']":'2.5',
                "['result-rating', 'three']":'3',
                "['result-rating', 'three', 'half']":'3.5',
                "['result-rating', 'four']":'4',
                "['result-rating', 'four', 'half']":'4.5',
                "['result-rating', 'five']":'5',
             }
        return switcher.get(i,"Invalid day of week")
ratin("['result-rating', 'five']")

In [4]:
import re
def parse(full_url):

               
    df = pd.DataFrame(columns = 
         ['businessname', 'rating', 'ratnumber', 'comment'])

    headers = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
    response=requests.get('https://www.yellowpages.com/los-angeles-ca/restaurant',headers=headers)
    soup=BeautifulSoup(response.content,'lxml')
    for item in soup.select('.v-card'):
        try:
              businessname=item.select('.business-name')[0].get_text()
        except: 
              businessname=None
        try:
           
              rating= item.select('.rating div')[0].get('class')
        except: 
              rating=None
     
        try:
           
              ratnumber= item.select('.rating div span')[0].get_text()
        except: 
              rating=None
        try:
              comment = item.find('p', {'class': 'body with-avatar'}).text.replace('\n', '')
        except: 
              comment=None
        if (not  rating) or  (not comment) :
            rating=None  
        else :
            df = df.append({'businessname': businessname, 'rating': rating, 'ratnumber': ratnumber, 'comment': comment}, ignore_index=True)
    return df



In [6]:
base_url = 'https://www.yellowpages.com/los-angeles-ca/restaurant'
all_reviews_df = pd.DataFrame(columns = ['businessname', 'rating', 'ratnumber', 'comment'])
num_reviews = 1
# you can adjust this number on how many reviews you which to scrape
while num_reviews < 4:  
    
    full_url = base_url + str(num_reviews)
    
    get_url = requests.get(full_url, timeout=5)  
        
    partial_reviews_df = parse(get_url)   
    all_reviews_df = all_reviews_df.append(
                     partial_reviews_df, ignore_index=True) 
    
    num_reviews += 1

In [8]:
all_reviews_df.to_csv('yellowpage_scrape.csv')

In [9]:
with open('yellowpage_scrape.csv') as f:
    df = pd.read_csv(f)
f.close()

In [10]:
for col in df.columns:
    print(col, df[col].isnull().sum())

Unnamed: 0 0
businessname 0
rating 0
ratnumber 0
comment 0


In [11]:
rws = df.loc[:, ['rating', 'comment']]

In [12]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [13]:
rws['no_contract'] = rws['comment'].apply(lambda x: [contractions.fix(word) for word in x.split()])
rws.head()

,rating,comment,no_contract
0,"['result-rating', 'four', 'half']",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place., Food, was, delicious, and, the, stuff, very, friendly!, I, loved, the, place, and..."
1,"['result-rating', 'four']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,..."
2,"['result-rating', 'four', 'half']",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here!, The, food, changed, into, excel..."
3,"['result-rating', 'four', 'half']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,..."
4,"['result-rating', 'four', 'half']",The staff were very welcoming and friendly. The traditional Middle Eastern cuisine was excellent...,"[The, staff, were, very, welcoming, and, friendly., The, traditional, Middle, Eastern, cuisine, ..."


In [15]:
rws['rating_description_str'] = [' '.join(map(str, l)) for l in rws['no_contract']]
rws.head()

,rating,comment,no_contract,rating_description_str
0,"['result-rating', 'four', 'half']",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place., Food, was, delicious, and, the, stuff, very, friendly!, I, loved, the, place, and...",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...
1,"['result-rating', 'four']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of..."
2,"['result-rating', 'four', 'half']",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here!, The, food, changed, into, excel...",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...
3,"['result-rating', 'four', 'half']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of..."
4,"['result-rating', 'four', 'half']",The staff were very welcoming and friendly. The traditional Middle Eastern cuisine was excellent...,"[The, staff, were, very, welcoming, and, friendly., The, traditional, Middle, Eastern, cuisine, ...",The staff were very welcoming and friendly. The traditional Middle Eastern cuisine was excellent...


In [16]:
rws['tokenized'] = rws['rating_description_str'].apply(word_tokenize)
rws.head()

,rating,comment,no_contract,rating_description_str,tokenized
0,"['result-rating', 'four', 'half']",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place., Food, was, delicious, and, the, stuff, very, friendly!, I, loved, the, place, and...",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place, ., Food, was, delicious, and, the, stuff, very, friendly, !, I, loved, the, place,..."
1,"['result-rating', 'four']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis..."
2,"['result-rating', 'four', 'half']",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here!, The, food, changed, into, excel...",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here, !, The, food, changed, into, exc..."
3,"['result-rating', 'four', 'half']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis..."
4,"['result-rating', 'four', 'half']",The staff were very welcoming and friendly. The traditional Middle Eastern cuisine was excellent...,"[The, staff, were, very, welcoming, and, friendly., The, traditional, Middle, Eastern, cuisine, ...",The staff were very welcoming and friendly. The traditional Middle Eastern cuisine was excellent...,"[The, staff, were, very, welcoming, and, friendly, ., The, traditional, Middle, Eastern, cuisine..."


In [17]:
rws['lower'] = rws['tokenized'].apply(lambda x: [word.lower() for word in x])
rws.head()

,rating,comment,no_contract,rating_description_str,tokenized,lower
0,"['result-rating', 'four', 'half']",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place., Food, was, delicious, and, the, stuff, very, friendly!, I, loved, the, place, and...",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place, ., Food, was, delicious, and, the, stuff, very, friendly, !, I, loved, the, place,...","[nice, place, ., food, was, delicious, and, the, stuff, very, friendly, !, i, loved, the, place,..."
1,"['result-rating', 'four']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis..."
2,"['result-rating', 'four', 'half']",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here!, The, food, changed, into, excel...",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here, !, The, food, changed, into, exc...","[me, and, my, buddies, had, a, high-quality, time, right, here, !, the, food, changed, into, exc..."
3,"['result-rating', 'four', 'half']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis..."
4,"['result-rating', 'four', 'half']",The staff were very welcoming and friendly. The traditional Middle Eastern cuisine was excellent...,"[The, staff, were, very, welcoming, and, friendly., The, traditional, Middle, Eastern, cuisine, ...",The staff were very welcoming and friendly. The traditional Middle Eastern cuisine was excellent...,"[The, staff, were, very, welcoming, and, friendly, ., The, traditional, Middle, Eastern, cuisine...","[the, staff, were, very, welcoming, and, friendly, ., the, traditional, middle, eastern, cuisine..."


In [18]:
punc = string.punctuation
rws['no_punc'] = rws['lower'].apply(lambda x: [word for word in x if word not in punc])
rws.head()

,rating,comment,no_contract,rating_description_str,tokenized,lower,no_punc
0,"['result-rating', 'four', 'half']",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place., Food, was, delicious, and, the, stuff, very, friendly!, I, loved, the, place, and...",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place, ., Food, was, delicious, and, the, stuff, very, friendly, !, I, loved, the, place,...","[nice, place, ., food, was, delicious, and, the, stuff, very, friendly, !, i, loved, the, place,...","[nice, place, food, was, delicious, and, the, stuff, very, friendly, i, loved, the, place, and, ..."
1,"['result-rating', 'four']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, it, was, the, worst, of, times, it, was, the, age, of, wisdom, i..."
2,"['result-rating', 'four', 'half']",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here!, The, food, changed, into, excel...",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here, !, The, food, changed, into, exc...","[me, and, my, buddies, had, a, high-quality, time, right, here, !, the, food, changed, into, exc...","[me, and, my, buddies, had, a, high-quality, time, right, here, the, food, changed, into, excell..."
3,"['result-rating', 'four', 'half']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, it, was, the, worst, of, times, it, was, the, age, of, wisdom, i..."
4,"['result-rating', 'four', 'half']",The staff were very welcoming and friendly. The traditional Middle Eastern cuisine was excellent...,"[The, staff, were, very, welcoming, and, friendly., The, traditional, Middle, Eastern, cuisine, ...",The staff were very welcoming and friendly. The traditional Middle Eastern cuisine was excellent...,"[The, staff, were, very, welcoming, and, friendly, ., The, traditional, Middle, Eastern, cuisine...","[the, staff, were, very, welcoming, and, friendly, ., the, traditional, middle, eastern, cuisine...","[the, staff, were, very, welcoming, and, friendly, the, traditional, middle, eastern, cuisine, w..."


In [19]:
stop_words = set(stopwords.words('english'))
rws['stopwords_removed'] = rws['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])
rws.head()

,rating,comment,no_contract,rating_description_str,tokenized,lower,no_punc,stopwords_removed
0,"['result-rating', 'four', 'half']",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place., Food, was, delicious, and, the, stuff, very, friendly!, I, loved, the, place, and...",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place, ., Food, was, delicious, and, the, stuff, very, friendly, !, I, loved, the, place,...","[nice, place, ., food, was, delicious, and, the, stuff, very, friendly, !, i, loved, the, place,...","[nice, place, food, was, delicious, and, the, stuff, very, friendly, i, loved, the, place, and, ...","[nice, place, food, delicious, stuff, friendly, loved, place, come, back, chance]"
1,"['result-rating', 'four']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, it, was, the, worst, of, times, it, was, the, age, of, wisdom, i...","[best, times, worst, times, age, wisdom, age, foolishness, ...]"
2,"['result-rating', 'four', 'half']",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here!, The, food, changed, into, excel...",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here, !, The, food, changed, into, exc...","[me, and, my, buddies, had, a, high-quality, time, right, here, !, the, food, changed, into, exc...","[me, and, my, buddies, had, a, high-quality, time, right, here, the, food, changed, into, excell...","[buddies, high-quality, time, right, food, changed, excellent, service, high, exceptional]"
3,"['result-rating', 'four', 'half']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, it, was, the, worst, of, times, it, was, the, age, of, wisdom, i...","[best, times, worst, times, age, wisdom, age, foolishness, ...]"
4,"['result-rating', 'four', 'half']",The staff were very welcoming and friendly. The traditional Middle Eastern cuisine was excellent...,"[The, staff, were, very, welcoming, and, friendly., The, traditional, Middle, Eastern, cuisine, ...",The staff were very welcoming and friendly. The traditional Middle Eastern cuisine was excellent...,"[The, staff, were, very, welcoming, and, friendly, ., The, traditional, Middle, Eastern, cuisine...","[the, staff, were, very, welcoming, and, friendly, ., the, traditional, middle, eastern, cuisine...","[the, staff, were, very, welcoming, and, friendly, the, traditional, middle, eastern, cuisine, w...","[staff, welcoming, friendly, traditional, middle, eastern, cuisine, excellent, strongly, recommend]"


In [20]:
rws['pos_tags'] = rws['stopwords_removed'].apply(nltk.tag.pos_tag)
rws.head()

,rating,comment,no_contract,rating_description_str,tokenized,lower,no_punc,stopwords_removed,pos_tags
0,"['result-rating', 'four', 'half']",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place., Food, was, delicious, and, the, stuff, very, friendly!, I, loved, the, place, and...",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place, ., Food, was, delicious, and, the, stuff, very, friendly, !, I, loved, the, place,...","[nice, place, ., food, was, delicious, and, the, stuff, very, friendly, !, i, loved, the, place,...","[nice, place, food, was, delicious, and, the, stuff, very, friendly, i, loved, the, place, and, ...","[nice, place, food, delicious, stuff, friendly, loved, place, come, back, chance]","[(nice, JJ), (place, NN), (food, NN), (delicious, JJ), (stuff, NN), (friendly, RB), (loved, VBD)..."
1,"['result-rating', 'four']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, it, was, the, worst, of, times, it, was, the, age, of, wisdom, i...","[best, times, worst, times, age, wisdom, age, foolishness, ...]","[(best, RBS), (times, NNS), (worst, RB), (times, NNS), (age, NN), (wisdom, NN), (age, NN), (fool..."
2,"['result-rating', 'four', 'half']",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here!, The, food, changed, into, excel...",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here, !, The, food, changed, into, exc...","[me, and, my, buddies, had, a, high-quality, time, right, here, !, the, food, changed, into, exc...","[me, and, my, buddies, had, a, high-quality, time, right, here, the, food, changed, into, excell...","[buddies, high-quality, time, right, food, changed, excellent, service, high, exceptional]","[(buddies, NNS), (high-quality, JJ), (time, NN), (right, JJ), (food, NN), (changed, VBD), (excel..."
3,"['result-rating', 'four', 'half']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, it, was, the, worst, of, times, it, was, the, age, of, wisdom, i...","[best, times, worst, times, age, wisdom, age, foolishness, ...]","[(best, RBS), (times, NNS), (worst, RB), (times, NNS), (age, NN), (wisdom, NN), (age, NN), (fool..."
4,"['result-rating', 'four', 'half']",The staff were very welcoming and friendly. The traditional Middle Eastern cuisine was excellent...,"[The, staff, were, very, welcoming, and, friendly., The, traditional, Middle, Eastern, cuisine, ...",The staff were very welcoming and friendly. The traditional Middle Eastern cuisine was excellent...,"[The, staff, were, very, welcoming, and, friendly, ., The, traditional, Middle, Eastern, cuisine...","[the, staff, were, very, welcoming, and, friendly, ., the, traditional, middle, eastern, cuisine...","[the, staff, were, very, welcoming, and, friendly, t

In [21]:
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
rws['wordnet_pos'] = rws['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])
rws.head()

,rating,comment,no_contract,rating_description_str,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos
0,"['result-rating', 'four', 'half']",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place., Food, was, delicious, and, the, stuff, very, friendly!, I, loved, the, place, and...",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place, ., Food, was, delicious, and, the, stuff, very, friendly, !, I, loved, the, place,...","[nice, place, ., food, was, delicious, and, the, stuff, very, friendly, !, i, loved, the, place,...","[nice, place, food, was, delicious, and, the, stuff, very, friendly, i, loved, the, place, and, ...","[nice, place, food, delicious, stuff, friendly, loved, place, come, back, chance]","[(nice, JJ), (place, NN), (food, NN), (delicious, JJ), (stuff, NN), (friendly, RB), (loved, VBD)...","[(nice, a), (place, n), (food, n), (delicious, a), (stuff, n), (friendly, r), (loved, v), (place..."
1,"['result-rating', 'four']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, it, was, the, worst, of, times, it, was, the, age, of, wisdom, i...","[best, times, worst, times, age, wisdom, age, foolishness, ...]","[(best, RBS), (times, NNS), (worst, RB), (times, NNS), (age, NN), (wisdom, NN), (age, NN), (fool...","[(best, r), (times, n), (worst, r), (times, n), (age, n), (wisdom, n), (age, n), (foolishness, n..."
2,"['result-rating', 'four', 'half']",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here!, The, food, changed, into, excel...",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here, !, The, food, changed, into, exc...","[me, and, my, buddies, had, a, high-quality, time, right, here, !, the, food, changed, into, exc...","[me, and, my, buddies, had, a, high-quality, time, right, here, the, food, changed, into, excell...","[buddies, high-quality, time, right, food, changed, excellent, service, high, exceptional]","[(buddies, NNS), (high-quality, JJ), (time, NN), (right, JJ), (food, NN), (changed, VBD), (excel...","[(buddies, n), (high-quality, a), (time, n), (right, a), (food, n), (changed, v), (excellent, a)..."
3,"['result-rating', 'four', 'half']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, it, was, the, worst, of, times, it, was, the, age, of, wisdom, i...","[best, times, worst, times, age, wisdom, age, foolishness, ...]","[(best, RBS), (times, NNS), (worst, RB), (times, NNS), (age, NN), (wisdom, NN), (age, NN), (fool...","[(best, r), (times, n), (worst, r), (times, n), (age, n), (wisdom, n), (age, n), (foolishness, n..."
4,"['result-rating', 'four', 'half']",The staff were very welcoming and friendly. The traditional Middle Eastern cuisine was excellent...,"[The, staff, were, very, welcoming, an

In [22]:
wnl = WordNetLemmatizer()
rws['lemmatized'] = rws['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])
rws.head()

,rating,comment,no_contract,rating_description_str,tokenized,lower,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,"['result-rating', 'four', 'half']",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place., Food, was, delicious, and, the, stuff, very, friendly!, I, loved, the, place, and...",Nice place. Food was delicious and the stuff very friendly! I loved the place and I will come ba...,"[Nice, place, ., Food, was, delicious, and, the, stuff, very, friendly, !, I, loved, the, place,...","[nice, place, ., food, was, delicious, and, the, stuff, very, friendly, !, i, loved, the, place,...","[nice, place, food, was, delicious, and, the, stuff, very, friendly, i, loved, the, place, and, ...","[nice, place, food, delicious, stuff, friendly, loved, place, come, back, chance]","[(nice, JJ), (place, NN), (food, NN), (delicious, JJ), (stuff, NN), (friendly, RB), (loved, VBD)...","[(nice, a), (place, n), (food, n), (delicious, a), (stuff, n), (friendly, r), (loved, v), (place...","[nice, place, food, delicious, stuff, friendly, love, place, come, back, chance]"
1,"['result-rating', 'four']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, it, was, the, worst, of, times, it, was, the, age, of, wisdom, i...","[best, times, worst, times, age, wisdom, age, foolishness, ...]","[(best, RBS), (times, NNS), (worst, RB), (times, NNS), (age, NN), (wisdom, NN), (age, NN), (fool...","[(best, r), (times, n), (worst, r), (times, n), (age, n), (wisdom, n), (age, n), (foolishness, n...","[best, time, worst, time, age, wisdom, age, foolishness, ...]"
2,"['result-rating', 'four', 'half']",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here!, The, food, changed, into, excel...",Me and my buddies had a high-quality time right here! The food changed into excellent and servic...,"[Me, and, my, buddies, had, a, high-quality, time, right, here, !, The, food, changed, into, exc...","[me, and, my, buddies, had, a, high-quality, time, right, here, !, the, food, changed, into, exc...","[me, and, my, buddies, had, a, high-quality, time, right, here, the, food, changed, into, excell...","[buddies, high-quality, time, right, food, changed, excellent, service, high, exceptional]","[(buddies, NNS), (high-quality, JJ), (time, NN), (right, JJ), (food, NN), (changed, VBD), (excel...","[(buddies, n), (high-quality, a), (time, n), (right, a), (food, n), (changed, v), (excellent, a)...","[buddy, high-quality, time, right, food, change, excellent, service, high, exceptional]"
3,"['result-rating', 'four', 'half']","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times,, it, was, the, worst, of, times,, it, was, the, age, of, wisdom,...","It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of...","[It, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, ,, it, was, the, worst, of, times, ,, it, was, the, age, of, wis...","[it, was, the, best, of, times, it, was, the, worst, of, times, it, was, the, age, of, wisdom, i...","[best, times, worst, times, age, wisdom, age, foolishness, ...]","[(best, RBS), (times, NNS), (worst, RB), (times, NNS), (age, NN), (wisdom, NN), (age, NN), (fool...","[(best, r), (times, n), (worst

In [23]:
rws.to_csv('yellowpage_scrape_clean.csv')